In [ ]:
## working in Spanish, but talks a lot of non-sense...

In [ ]:
#!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
#!pip install -q datasets bitsandbytes einops wandb

In [1]:
import torch
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id  = "aguila"

tokenizer = AutoTokenizer.from_pretrained(model_id)
generator = pipeline(
    "text-generation",
    #'conversational',
    model=model_id,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

A new version of the following files was downloaded from https://huggingface.co/projecte-aina/aguila-7b:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/projecte-aina/aguila-7b:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Loading checkpoint shards: 100%|█████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.45s/it]
The model 'RWForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'Camembert

In [ ]:
input_text = "El mercat del barri és fantàstic, hi pots trobar"

In [ ]:
generation = generator(
    input_text,
    do_sample=True,
    top_k=10,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
print(f"Result: {generation[0]['generated_text']}")

In [ ]:
input_text = "User: hola, cómo estás?\nMini:"

In [ ]:
generation = generator(
    input_text,
    do_sample=True,
    top_k=2,
    eos_token_id=tokenizer.eos_token_id,
)

In [ ]:
print(f"Result: {generation[0]['generated_text']}")

In [3]:
# Few Shots
input_text = """
Oració: Estoy bien, gracias.
Paráfrasis: Tú estás bien, yo también estoy bien. ¿Dónde vive?
----
Oració: no me sirve.
Paráfrasis: Tú no estás bien, yo estoy bien. ¿Dónde vive?
----
Oración: Estoy bien.
Paráfrasis: Tú estás bien, yo también estoy bien. ¿Dónde vive?
----
Oració: Estoy muy bien.
Paráfrasis:"""

In [4]:
%%time
generation = generator(
    input_text,
    do_sample=True,
    top_k=2,
    max_length=128,
    eos_token_id=tokenizer.eos_token_id,
)

CPU times: user 900 ms, sys: 114 ms, total: 1.01 s
Wall time: 1.04 s


In [5]:
print(f"Result: {generation[0]['generated_text']}")

Result: 
Oració: Estoy bien, gracias.
Paráfrasis: Tú estás bien, yo también estoy bien. ¿Dónde vive?
----
Oració: no me sirve.
Paráfrasis: Tú no estás bien, yo estoy bien. ¿Dónde vive?
----
Oración: Estoy bien.
Paráfrasis: Tú estás bien, yo también estoy bien. ¿Dónde vive?
----
Oració: Estoy muy bien.
Paráfrasis: Tú estás muy bien, yo también estoy bien. ¿Dónde vive?
----


In [6]:
%%time

sequences = generator(
    input_text,
    #max_length=128,
    max_new_tokens=64,
    do_sample=True,
    temperature=0.1,
    top_p=0.7,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
print(f"Result: {sequences[0]['generated_text']}")

Result: 
Oració: Estoy bien, gracias.
Paráfrasis: Tú estás bien, yo también estoy bien. ¿Dónde vive?
----
Oració: no me sirve.
Paráfrasis: Tú no estás bien, yo estoy bien. ¿Dónde vive?
----
Oración: Estoy bien.
Paráfrasis: Tú estás bien, yo también estoy bien. ¿Dónde vive?
----
Oració: Estoy muy bien.
Paráfrasis: Tú estás muy bien, yo también estoy muy bien. ¿Dónde vive?
----
Oració: Estoy bien.
Paráfrasis: Tú estás muy bien, yo también estoy muy bien. ¿Dónde vive?
----
Oració: Estoy bien.
Paráfras
CPU times: user 3.24 s, sys: 16.1 ms, total: 3.26 s
Wall time: 3.26 s
